### Keras로 RNN 구현하기

In [2]:
from keras.layers import SimpleRNN

# model.add(SimpleRNN(hidden_units))

- hidden_units = 은닉 상태의 크기를 정의(batch_size)
    - 메모리 셀이 다음 시점의 메모리 셀과 출력층으로 보내는 값의 크기(output_dim)

    - RNN의 용량을 늘린다고 보면 되며, 중소형 모델의 경우 보통 128, 256, 512, 1024 등의 값을 가짐

- timesteps = 입력 시퀀스의 길이(input_length), 시점의 수(timeseries)

- input_dim = 입력의 크기(Dimensionality of word representation)

In [ ]:
# 추가 인자를 사용할 때
# model.add(SimpleRNN(hidden_units, input_shape=(timesteps, input_dim)))

# 다른 표기
# model.add(SimpleRNN(hidden_units, input_length=M, input_dim=N))

In [3]:
from keras.models import Sequential
from keras.layers import SimpleRNN 

# batch_size를 정의하지 않았을 때
model = Sequential()
model.add(SimpleRNN(3, input_shape=(2,10)))
# model.add(SimpleRNN(3, input_length=2, input_dim=10)) <-위와 같은 뜻
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 3)                 42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


- 위의 결과에서 출력값이 (batch_size, output_dim)크기의 2D 텐서일 때, output_dim은 hidden_units의 값인 3

- 이 경우 batch_size를 현 단계에서 알 수 x -> (None,3)

In [4]:
# batch_size를 정의하지 않았을 때
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10)))
# model.add(SimpleRNN(3, input_length=2, input_dim=10)) <-위와 같은 뜻
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (8, 3)                    42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


- 위의 경우에는 batch_size를 미리 8로 정의 했기에 출력의 크기가 (8,3)이 되는 것을 확인 해 볼 수 있음

In [5]:
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10), return_sequences=True))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_2 (SimpleRNN)    (8, 2, 3)                 42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


- return_sequences = True를 기재함으로 출력값으로 (batch_size, timesteps, output_dim) 크기의 3D 텐서를 리턴하는 모델 생성

### 파이썬으로 RNN 구현

- Ht = tanh(Wx x Xt + Wh x H(t-1) + b )

In [6]:
# pseudocode. 
'''
hidden_state_t = 0 # 초기 은닉 상태를 0(벡터)로 초기화
for input_t in input_length: # 각 시점마다 입력을 받는다.
    output_t = tanh(input_t, hidden_state_t) # 각 시점에 대해서 입력과 은닉 상태를 가지고 연산
    hidden_state_t = output_t # 계산 결과는 현재 시점의 은닉 상태가 된다.
'''

'\nhidden_state_t = 0 # 초기 은닉 상태를 0(벡터)로 초기화\nfor input_t in input_length: # 각 시점마다 입력을 받는다.\n    output_t = tanh(input_t, hidden_state_t) # 각 시점에 대해서 입력과 은닉 상태를 가지고 연산\n    hidden_state_t = output_t # 계산 결과는 현재 시점의 은닉 상태가 된다.\n'

In [7]:
import numpy as np 

timesteps = 10
input_dim = 4 
hidden_units = 8

In [11]:
# 2D 입력 텐서 생성
inputs = np.random.random((timesteps, input_dim))
print(inputs)

[[0.71072179 0.32029377 0.87679573 0.11836379]
 [0.13137235 0.45305515 0.72336449 0.46044652]
 [0.26463364 0.9135067  0.2381111  0.89380745]
 [0.09226061 0.0529971  0.2089529  0.57551961]
 [0.44166061 0.62480188 0.9919051  0.58835693]
 [0.21591783 0.39786446 0.54834123 0.04378464]
 [0.39075854 0.92740593 0.57468076 0.21537265]
 [0.49634097 0.52986372 0.08590758 0.14702445]
 [0.84874538 0.59172122 0.88595492 0.65169358]
 [0.62202523 0.02104251 0.60368395 0.63626015]]


In [10]:
# 초기 은닉 상태는 0(벡터)로 초기화
hidden_state_t = np.zeros((hidden_units))
print(hidden_state_t)

[0. 0. 0. 0. 0. 0. 0. 0.]


In [12]:
Wx = np.random.random((hidden_units, input_dim)) # 입력에 대한 가중치
Wh = np.random.random((hidden_units, hidden_units)) # 은닉 상태에 대한 가중치
b= np.random.random((hidden_units)) # 편향

print(np.shape(Wx))
print(np.shape(Wh))
print(np.shape(b))

(8, 4)
(8, 8)
(8,)


In [13]:
total_hidden_states = []

for input_t in inputs:
    
    # Wx * Xt + Wh * Ht-1 + b
    output_t = np.tanh(np.dot(Wx,input_t) + np.dot(Wh,hidden_state_t)+b)
    
    # 각 시점 t별 메모리 셀의 출력의 크기는 (timestep t, output_dim)
    # 각 시점의 은닉 상태의 값을 계속해서 누적
    total_hidden_states.append(list(output_t))
    hidden_state_t = output_t
    
# 출력시 값을 깔끔하게 해주는 용도
total_hidden_states = np.stack(total_hidden_states, axis = 0)

print('모든 시점의 은닉 상태: ')
print(total_hidden_states)
    

모든 시점의 은닉 상태: 
[[0.88936824 0.94025144 0.94282667 0.85031818 0.65534797 0.8788657
  0.53990627 0.97176058]
 [0.99973773 0.99878298 0.99997403 0.99889139 0.99969364 0.99882803
  0.99916972 0.99993218]
 [0.99997709 0.99973617 0.99999652 0.99993443 0.99992385 0.99978801
  0.99989574 0.99998574]
 [0.99984434 0.99918493 0.99999053 0.99947174 0.99983187 0.99932691
  0.99952989 0.99993054]
 [0.99996851 0.99990261 0.99999809 0.9999367  0.99994931 0.99985908
  0.99986547 0.99999481]
 [0.99992708 0.99947463 0.99998957 0.99978779 0.99974471 0.9991281
  0.99963596 0.99997819]
 [0.99997866 0.99976826 0.99999482 0.99994796 0.99983839 0.99961258
  0.99985545 0.99999263]
 [0.99995018 0.99949753 0.99999074 0.99987956 0.99961887 0.99938507
  0.99966699 0.99997517]
 [0.9999749  0.99995053 0.99999897 0.9999664  0.99994793 0.99993638
  0.99986774 0.99999646]
 [0.99990009 0.99984068 0.99999757 0.99982341 0.99991144 0.99984562
  0.99964352 0.99998224]]


### Deep Recurrent Neural Network

In [16]:
model = Sequential()
# 첫 번째 은닉층은 다음 은닉층이 존재함으로 return_sequences = True로 하여 모든 시점에 대해서 은닉 상태 값을 다음 은닉층으로 보내줌
model.add(SimpleRNN(hidden_units, input_length=10, input_dim=5, return_sequences=True))
model.add(SimpleRNN(hidden_units, return_sequences=True))
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_8 (SimpleRNN)    (None, 10, 8)             112       
                                                                 
 simple_rnn_9 (SimpleRNN)    (None, 10, 8)             136       
                                                                 
Total params: 248
Trainable params: 248
Non-trainable params: 0
_________________________________________________________________


### (Deep) Bidirectional Recurrent Neural Network

In [17]:
from keras.layers import Bidirectional
timesteps = 10
input_dim = 5

# 간단한 양방향 RNN
model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True), input_shape=(timesteps,input_dim)))
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_5 (Bidirectio  (None, 10, 16)           224       
 nal)                                                            
                                                                 
Total params: 224
Trainable params: 224
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Deep Bidirectional RNN
model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_units,return_sequences=True), input_shape=(timesteps, input_dim)))
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True)))
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True)))
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True)))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_1 (Bidirectio  (None, 10, 16)           224       
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 10, 16)           400       
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 10, 16)           400       
 nal)                                                            
                                                                 
 bidirectional_4 (Bidirectio  (None, 10, 16)           400       
 nal)                                                            
                                                                 
Total params: 1,424
Trainable params: 1,424
Non-traina